<a href="https://colab.research.google.com/github/jithsg/Mistral-7B-eg/blob/main/mistral_7b_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y tensorflow --quiet
!pip install ludwig --quiet
!pip install ludwig[llm] --quiet

In [2]:
import locale; locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel

In [3]:
from google.colab import data_table; data_table.enable_dataframe_formatter()
import numpy as np; np.random.seed(123)
import pandas as pd

df = pd.read_json("https://raw.githubusercontent.com/sahil280114/codealpaca/master/data/code_alpaca_20k.json")

# We're going to create a new column called `split` where:
# 90% will be assigned a value of 0 -> train set
# 5% will be assigned a value of 1 -> validation set
# 5% will be assigned a value of 2 -> test set
# Calculate the number of rows for each split value
total_rows = len(df)
split_0_count = int(total_rows * 0.9)
split_1_count = int(total_rows * 0.05)
split_2_count = total_rows - split_0_count - split_1_count

# Create an array with split values based on the counts
split_values = np.concatenate([
    np.zeros(split_0_count),
    np.ones(split_1_count),
    np.full(split_2_count, 2)
])

# Shuffle the array to ensure randomness
np.random.shuffle(split_values)

# Add the 'split' column to the DataFrame
df['split'] = split_values
df['split'] = df['split'].astype(int)

# For this webinar, we will just 500 rows of this dataset.
df = df.head(n=500)

In [4]:
df

,instruction,input,output,split
0,Create an array of length 5 which contains all...,,"arr = [2, 4, 6, 8, 10]",0
1,Formulate an equation to calculate the height ...,,Height of triangle = opposite side length * si...,0
2,Write a replace method for a string class whic...,"string = ""Hello World!""\nreplace_with = ""Greet...","def replace(self, replace_with):\n new_stri...",0
3,Create an array of length 15 containing number...,,"arr = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33...",0
4,Write a function to find the number of distinc...,"matrix = [[1, 0, 0],\n [1, 0, 1],\n ...",def find_num_distinct_states(matrix):\n sta...,0
...,...,...,...,...
495,Create a function in Python to evaluate if a g...,,def is_prime(num):\n if num <= 1: \n ...,0
496,Construct a query in PostgreSQL to delete all ...,,DELETE FROM table_name;,0
497,Construct a query in PostgreSQL to find the na...,,"SELECT name, email\nFROM users\nWHERE email LI...",0
498,Create a program in JavaScript to check if two...,,// Function to check if two strings are anagra...,0


In [5]:
num_self_sufficient = (df['input'] == '').sum()
num_need_context = df.shape[0] - num_self_sufficient

# We are only using 100 rows of this dataset for this webinar
print(f"Total number of examples in the dataset: {df.shape[0]}")

print(f"% of examples that are self-sufficient: {round(num_self_sufficient/df.shape[0] * 100, 2)}")
print(f"% of examples that are need additional context: {round(num_need_context/df.shape[0] * 100, 2)}")

Total number of examples in the dataset: 500
% of examples that are self-sufficient: 46.6
% of examples that are need additional context: 53.4


In [7]:
from transformers import AutoTokenizer
import numpy as np

def calculate_distribution(data_dict):
    result = {}

    for key, values in data_dict.items():
        values = np.array(values)
        result[key] = {
            'average': int(np.mean(values)),
            'min': np.min(values),
            'max': np.max(values),
            'median': np.median(values),
            '75th_percentile': int(np.percentile(values, 75)),
            '90th_percentile': int(np.percentile(values, 90)),
            '95th_percentile': int(np.percentile(values, 95)),
            '99th_percentile': int(np.percentile(values, 99))
        }

    return result

tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')

token_counts = {
    "instruction": [],
    "input": [],
    "output": [],
    "total": []
}

for index, row in df.iterrows():
    instruction_col_tokens = len(tokenizer.tokenize(row['instruction']))
    input_col_tokens = len(tokenizer.tokenize(row['input']))
    output_col_tokens = len(tokenizer.tokenize(row['output']))
    total = instruction_col_tokens + input_col_tokens + output_col_tokens

    token_counts['instruction'].append(instruction_col_tokens)
    token_counts['input'].append(input_col_tokens)
    token_counts['output'].append(output_col_tokens)
    token_counts['total'].append(total)

token_distribution = calculate_distribution(token_counts)
token_distribution = pd.DataFrame(token_distribution)
token_distribution

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

,instruction,input,output,total
average,15.0,10.0,66.0,92.0
min,5.0,0.0,2.0,12.0
max,39.0,120.0,327.0,359.0
median,15.0,5.0,52.5,78.0
75th_percentile,18.0,16.0,96.0,122.0
90th_percentile,21.0,26.0,139.0,164.0
95th_percentile,23.0,34.0,164.0,196.0
99th_percentile,27.0,66.0,254.0,295.0


In [9]:
qlora_fine_tuning_config = yaml.safe_load(
"""
model_type: llm
base_model: mistralai/Mistral-7B-Instruct-v0.1

input_features:
  - name: instruction
    type: text

output_features:
  - name: output
    type: text

prompt:
  template: >-
    Below is an instruction that describes a task, paired with an input
    that provides further context. Write a response that appropriately
    completes the request.

    ### Instruction: {instruction}

    ### Input: {input}

    ### Response:

generation:
  temperature: 0.1
  max_new_tokens: 64

adapter:
  type: lora

quantization:
  bits: 4

preprocessing:
  global_max_sequence_length: 512
  split:
    type: random
    probabilities:
    - 0.95
    - 0
    - 0.05

trainer:
  type: finetune
  epochs: 1 # Typically, you want to set this to 3 epochs for instruction fine-tuning
  batch_size: 1
  eval_batch_size: 2
  optimizer:
    type: paged_adam
  gradient_accumulation_steps: 16
  learning_rate: 0.0004
  learning_rate_scheduler:
    decay: cosine
    warmup_fraction: 0.03
"""
)

model = LudwigModel(config=qlora_fine_tuning_config, logging_level=logging.INFO)
results = model.train(dataset=df)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

INFO:ludwig.trainers.trainer:
Running evaluation for step: 475, epoch: 1


Evaluation test : 100%|██████████| 13/13 [00:03<00:00,  3.89it/s]

INFO:ludwig.trainers.trainer_llm:Input: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction: Describe the steps of creating an application using the Flask framework.
### Input: 
### Response:
INFO:ludwig.trainers.trainer_llm:Output: Question: a exampleal will how process that andired with a example that is the information.
 a program to describesately addresseses the task.
 Instruction: Write your weather to the a app. the Flask framework.
### Input: Fl1
 Response: Flps to create an Flask application::ing the Flask module, creating a instance of theask, defining routes endpoint template,, defining defining a route function the application. mapsers the template template. Here route step is to define a function that the route for handle the the route. this file include database database database query or processing an function service, or a, or. Finally the code is be


Training: 100%|██████████| 475/475 [03:07<00:00,  2.53it/s, loss=0.0183]


INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ FINISHED │
INFO:ludwig.utils.print_utils:╘══════════╛
INFO:ludwig.utils.print_utils:


In [10]:
test_examples = pd.DataFrame([
      {
            "instruction": "Create an array of length 5 which contains all even numbers between 1 and 10.",
            "input": "",
      },
      {
            "instruction": "Create an array of length 15 containing numbers divisible by 3 up to 45.",
            "input": "",
      },
      {
            "instruction": "Create a nested loop to print every combination of numbers between 0-9",
            "input": "",
      },
      {
            "instruction": "Generate a function that computes the sum of the numbers in a given list",
            "input": "",
      },
      {
            "instruction": "Create a class to store student names, ages and grades.",
            "input": "",
      },
      {
            "instruction": "Print out the values in the following dictionary.",
            "input": "my_dict = {\n  'name': 'John Doe',\n  'age': 32,\n  'city': 'New York'\n}",
      },
])

predictions = model.predict(test_examples, generation_config={"max_new_tokens": 64, "temperature": 0.1})[0]
for input_with_prediction in zip(test_examples['instruction'], test_examples['input'], predictions['output_response']):
  print(f"Instruction: {input_with_prediction[0]}")
  print(f"Input: {input_with_prediction[1]}")
  print(f"Generated Output: {input_with_prediction[2][0]}")
  print("\n\n")

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of mistralai/Mistral-7B-Instruct-v0.1 tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prediction: 100%|██████████| 1/1 [00:20<00:00, 20.68s/it]

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of mistralai/Mistral-7B-Instruct-v0.1 tokenizer


/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
INFO:ludwig.api:Finished predicting in: 21.75s.


Instruction: Create an array of length 5 which contains all even numbers between 1 and 10.
Input: 
Generated Output: let evenNumbers = [2, 4, 6, 8, 10];



Instruction: Create an array of length 15 containing numbers divisible by 3 up to 45.
Input: 
Generated Output: 
let arr = [];
for (let i = 0; i < 15; i++) {
  if (i % 3 === 0) {
    arr.push(i);
  }
}
console.log(arr);



Instruction: Create a nested loop to print every combination of numbers between 0-9
Input: 
Generated Output: for i in range(10):
    for j in range(10):
        print(i, j)



Instruction: Generate a function that computes the sum of the numbers in a given list
Input: 
Generated Output: def sum_list(lst):
    total = 0
    for num in lst:
        total += num
    return total



Instruction: Create a class to store student names, ages and grades.
Input: 
Generated Output: class Student:
    def __init__(self, name, age, grade):
        self.name = name
        self.age = age
        self.grade = grade



Instruct